
<h2>Machine learning</h2>
<h3>Data clustering algorithms implementation</h3>

This notebook presents the implementation and application of some traditional machine learning clustering algorithms.
In the following cell I make the necessary imports, including tensorflow, which provides us with the image set on which we experiment.



In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
from math import sqrt, floor, log
import cmath
import sklearn.metrics as metrics
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.4.1


Importing the dataset. I won't use the labels, since we're practicing unsupervised learning. 

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Below I store the names of each clothing category in a list, so as to present the score values for each class seperately when the time to print stats and results comes.

In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Normalising the dataset.

In [4]:
# data normalisation by dividing with 255, which is the maximum pixel value and corresponds to deep black

train_images = train_images / 255.0
test_images = test_images / 255.0

I extract a small subset of the initial data (1000 images), because the algorithms would take very long to finish for the initial whole set of data. I make sure the subset has evenly distributed data between each category.

In [5]:
train_subset_size = 1000
train_subset_images = []
train_subset_labels = []

for category_index in range(10):
    remaining_from_category = train_subset_size / 10 # we need equally distributed data per category
    image_index = 0
    
    while remaining_from_category > 0 :
        if train_labels[image_index] == category_index:
            train_subset_images.append(train_images[image_index])
            train_subset_labels.append(train_labels[image_index])
            remaining_from_category -= 1
        image_index += 1
        


In [6]:
# R1 dataset : 28x28 = 784 feature vector

train_subset_images = np.array(train_subset_images)
train_subset_labels = np.array(train_subset_labels)

nsamples, nx, ny = train_subset_images.shape 
train_subset_images_R1 = train_subset_images.reshape((nsamples,nx*ny))


In [7]:
print(train_subset_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

Declaring a euclidean distance calculation function.

In [8]:
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

Trying out euclidean_distance()

In [9]:
print(euclidean_distance(np.array([-7, -4]), np.array([17, 6.5])))

26.196373794859472


Bellow I implement the k_means algorithm from scratch. I create a model with a predict() function which if parameterised by the vector data, it will cluster them without the need for training (unsupervised learning).

In [10]:
# max_iter is the maximum iterations that we allow in case we don't converge earlier
# k = 10 for k clusters (since the clothing categories are actually 10)

class k_means:
    def __init__(self, K = 10, max_iter = 300, tolerance = 0.001):
        self.K = K
        self.max_iter = max_iter
        self.tolerance = tolerance
        
        # list of sample indices for each cluster
        self.clusters = [[] for i in range(self.K)]
        
        # mean feature vector for each cluster
        self.centroids = []
        
    def predict(self, X):
        self.X = X
        print(X.shape)
        self.n_samples, self.n_features = X.shape
        print("Starting algorithm")
        # initialise centroids (10 of the feature vectors will be picked randomly)
        random_sample_indices = np.random.choice(self.n_samples, self.K, replace=False)
        self.centroids = [self.X[index] for index in random_sample_indices]
        
        # optimisation
        
        for i in range(self.max_iter):
            # update clusters
            self.clusters = self._create_clusters(self.centroids)
            
            # update centroids
            centroids_old = self.centroids # in order to check for convergence
            self.centroids = self._get_centroids(self.clusters)
            
            # check for convergence
            if self._is_converged(centroids_old, self.centroids):
                break
                
            print("Finished iteration", i)
            
        # return cluster labels
        return self.__get_cluster_labels(self.clusters)
    
    def __get_cluster_labels(self, clusters):
        labels = np.empty(self.n_samples)
        for cluster_index, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_index
                
        return labels
       
    # assign each sample to the cluster of the closest centroid
    def _create_clusters(self, centroids):
        clusters = [[] for i in range(self.K)]
        
        for index, sample in enumerate(self.X):
            centroid_index = self._closest_centroid(sample, centroids)
            clusters[centroid_index].append(index)
            
        return clusters
    
    # calculate the distances of the current sample from each centroid (according to L2 distance)
    def _closest_centroid(self, sample, centroids):
        distances = [euclidean_distance(sample, centroid) for centroid in centroids]
        closest_index = np.argmin(distances)
        return closest_index
    
    # calculate the centroids as the mean value of the samples in each cluster
    def _get_centroids(self, clusters):
        centroids = np.zeros((self.K, self.n_features))
        
        for cluster_index, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.X[cluster])
            centroids[cluster_index] = cluster_mean
            
        return centroids
   
    # check for convergence
    def _is_converged(self, centroids_old, centroids):
        distances = [euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        return sum(distances) <= self.tolerance

Testing the model on our dataset.

In [11]:
k_means_model = k_means(K = 10, max_iter = 300)
k_means_predictions = k_means_model.predict(train_subset_images_R1)

(1000, 784)
Starting algorithm
Finished iteration 0
Finished iteration 1
Finished iteration 2
Finished iteration 3
Finished iteration 4
Finished iteration 5
Finished iteration 6
Finished iteration 7
Finished iteration 8
Finished iteration 9
Finished iteration 10
Finished iteration 11
Finished iteration 12
Finished iteration 13
Finished iteration 14
Finished iteration 15
Finished iteration 16
Finished iteration 17
Finished iteration 18
Finished iteration 19
Finished iteration 20
Finished iteration 21
Finished iteration 22
Finished iteration 23
Finished iteration 24
Finished iteration 25
Finished iteration 26
Finished iteration 27
Finished iteration 28
Finished iteration 29
Finished iteration 30
Finished iteration 31
Finished iteration 32
Finished iteration 33
Finished iteration 34
Finished iteration 35
Finished iteration 36
Finished iteration 37
Finished iteration 38
Finished iteration 39
Finished iteration 40
Finished iteration 41
Finished iteration 42
Finished iteration 43
Finished it

In [12]:
# convert to integer array because the items will be used for indexing

k_means_predictions = k_means_predictions.astype(int)
print(k_means_predictions)

[0 7 5 3 6 6 1 3 5 3 0 5 5 6 0 5 0 0 0 5 7 5 9 6 7 9 5 3 0 4 4 6 5 6 3 9 0
 6 3 8 8 3 5 0 9 7 3 5 9 5 8 0 5 7 4 9 3 0 0 3 5 8 9 5 3 5 0 0 5 0 3 8 0 8
 5 4 0 6 0 0 5 5 5 0 0 6 0 3 6 8 7 0 7 9 0 6 7 0 3 3 8 8 8 9 8 8 9 5 8 8 5
 8 9 8 7 8 7 5 9 3 9 8 8 5 9 8 9 6 9 8 8 9 8 7 9 8 8 3 9 8 8 7 9 8 8 9 8 9
 7 8 9 9 9 8 9 8 8 7 8 8 8 9 4 5 8 8 8 9 8 5 9 9 9 8 9 9 8 8 8 8 7 9 9 8 5
 8 9 9 8 1 8 8 8 6 8 3 8 9 8 9 0 2 4 5 6 2 9 4 9 8 4 3 0 0 0 5 2 6 2 6 6 4
 0 5 5 4 0 4 0 4 8 0 9 0 9 9 9 5 4 4 1 5 9 5 3 8 4 5 5 0 6 6 0 2 5 0 4 4 2
 0 6 0 0 0 6 6 4 0 6 6 9 2 8 4 4 1 0 8 6 0 4 6 6 4 4 9 0 6 4 9 9 3 4 4 2 4
 9 0 4 0 8 3 3 9 3 6 5 5 5 9 6 3 7 9 5 9 3 6 8 9 8 7 5 7 9 9 5 6 7 4 5 9 7
 8 5 9 8 5 3 5 3 5 5 8 8 8 1 6 8 0 0 8 3 9 5 3 6 5 4 8 8 9 6 7 5 3 3 5 5 8
 3 5 7 8 8 7 9 9 9 5 8 3 5 3 3 9 0 3 6 8 6 7 8 3 3 8 7 7 5 3 7 5 0 5 0 3 6
 5 7 6 4 8 4 0 0 6 0 3 6 0 5 6 4 0 6 4 4 4 7 2 2 5 3 1 3 0 6 8 7 0 4 5 0 6
 8 0 4 7 3 0 6 4 4 4 6 4 3 5 4 6 2 4 4 0 4 4 9 0 6 5 8 9 3 0 3 0 6 3 8 6 6
 4 8 4 6 0 6 5 3 0 0 0 6 

Purity calculation function implementation. This function calculates purity for each seperate cluster.

In [13]:
def calculate_purity(predictions, number_of_classes, train_labels):
    
    cluster_labels = [[] for i in range(number_of_classes)]
    for i in range(len(predictions)):
        cluster_labels[predictions[i]].append(train_labels[i]) # get the actual categories for each cluster
      
    clusters_to_labels = []
    
    for i in range(len(cluster_labels)):
        most_occured = max(set(cluster_labels[i]), key=cluster_labels[i].count)
        if most_occured not in clusters_to_labels:
            clusters_to_labels.append(most_occured)
        else:
            while most_occured in clusters_to_labels:
                cluster_labels[i] = [x for x in cluster_labels[i] if x != most_occured]
                most_occured = max(set(cluster_labels[i]), key=cluster_labels[i].count)
            clusters_to_labels.append(most_occured)
                
    print(clusters_to_labels)
        
    purity_per_class = np.zeros((10,), dtype=float)
    items_per_class = np.zeros((10,), dtype=int)
    for i in range(len(predictions)):
        if clusters_to_labels[predictions[i]] == train_labels[i]:
            purity_per_class[train_labels[i]] += 1
        items_per_class[train_labels[i]] += 1
        
    for i in range(len(purity_per_class)):
        purity_per_class[i] = purity_per_class[i] / items_per_class[i]
        
    return purity_per_class
        

Implementing an f-measure calculation function. It calculates the f-measure for each cluster seperately and then print out the overall f-measure as well.

In [14]:
def calculate_F_measure(predictions, number_of_classes, train_labels):
    
    cluster_labels = [[] for i in range(number_of_classes)]
    for i in range(len(predictions)):
        cluster_labels[predictions[i]].append(train_labels[i]) # get the actual categories for each cluster
      
    clusters_to_labels = []
    for i in range(len(cluster_labels)):
        most_occured = max(set(cluster_labels[i]), key=cluster_labels[i].count)
        if most_occured not in clusters_to_labels:
            clusters_to_labels.append(most_occured)
        else:
            while most_occured in clusters_to_labels:
                cluster_labels[i] = [x for x in cluster_labels[i] if x != most_occured]
                most_occured = max(set(cluster_labels[i]), key=cluster_labels[i].count)
            clusters_to_labels.append(most_occured)
                
    print(clusters_to_labels)
    
    TP_per_cluster = np.zeros((10,), dtype=int)
    FP_per_cluster = np.zeros((10,), dtype=int)
    FN_per_cluster = np.zeros((10,), dtype=int)
    
    for cluster_index, cluster in enumerate(cluster_labels):
        for label in cluster:
            if label == clusters_to_labels[cluster_index]:
                TP_per_cluster[cluster_index] += 1
            else:
                FN_per_cluster[cluster_index] += 1
                false_predicted_cluster_index = clusters_to_labels.index(label)
                FP_per_cluster[false_predicted_cluster_index] += 1
                
    f_measure_per_cluster = np.zeros((10,), dtype=float)
    precision_measure_per_cluster = np.zeros((10,), dtype=float)
    recall_measure_per_cluster = np.zeros((10,), dtype=float)
    
    for i in range(10):
        precision_measure_per_cluster[i] = TP_per_cluster[i] / (TP_per_cluster[i] + FP_per_cluster[i])
        recall_measure_per_cluster[i] = TP_per_cluster[i] / (TP_per_cluster[i] + FN_per_cluster[i])
        f_measure_per_cluster[i] = 2*((precision_measure_per_cluster[i] * recall_measure_per_cluster[i]) / (precision_measure_per_cluster[i] + recall_measure_per_cluster[i]))
        print("Cluster", i ," f-measure:", f_measure_per_cluster[i])
        
    total_f_measure = np.sum(f_measure_per_cluster)
    print("Method total f-measure:", total_f_measure)
    

Printing out k-means results using the functions I implemented.

In [15]:
k_means_purity = calculate_purity(k_means_predictions, 10, train_subset_labels)
print("K_means with R1 purity \n")
for i in range(len(k_means_purity)):
    print(class_names[i] + " purity: " + str(k_means_purity[i]))

[0, 5, 2, 9, 4, 3, 8, 7, 1, 6]
K_means with R1 purity 

T-shirt/top purity: 0.25
Trouser purity: 0.48
Pullover purity: 0.08
Dress purity: 0.21
Coat purity: 0.2
Sandal purity: 0.38
Shirt purity: 0.14
Sneaker purity: 0.38
Bag purity: 0.16
Ankle boot purity: 0.22


Printing out overall purity as well.

In [16]:
purity = np.mean(k_means_purity)
print(purity)

0.25000000000000006


Printing f-measure results as well.

In [17]:
print("K_means with R1 f-measure \n")
calculate_F_measure(k_means_predictions, 10, train_subset_labels)

K_means with R1 f-measure 

[0, 5, 2, 9, 4, 3, 8, 7, 1, 6]
Cluster 0  f-measure: 0.26881720430107525
Cluster 1  f-measure: 0.5629629629629629
Cluster 2  f-measure: 0.17582417582417584
Cluster 3  f-measure: 0.24444444444444444
Cluster 4  f-measure: 0.3252032520325203
Cluster 5  f-measure: 0.25454545454545463
Cluster 6  f-measure: 0.17679558011049723
Cluster 7  f-measure: 0.42696629213483145
Cluster 8  f-measure: 0.43438914027149317
Cluster 9  f-measure: 0.18181818181818182
Method total f-measure: 3.0517666884456367


Creating a function which creates a histogram for each input image, based on the bins we set as parameter.

In [18]:
def histogram(images, bins):
    hist = np.zeros((len(images), bins))
    
    for image_index, image in enumerate(images):
        for row in range(len(image)):
            for column in range(len(image[0])):
                bin_index = floor(image[row][column] / (1/bins))
                if bin_index == bins:
                    bin_index = bins - 1
                hist[image_index][bin_index] += 1
                
    return hist          
            

The histogram dataset will be named R2.

In [19]:
R2 = histogram(train_subset_images, bins=32)

Printing out one histogram from the whole dataset.

In [20]:
print(R2[0])

[327.   4.   4.   0.   2.  10.  10.   3.   6.   9.   4.   5.   5.   3.
   7.   7.   5.   5.   1.   2.   5.   3.   1.   7.  38. 136.  87.  32.
  12.   9.  11.  24.]


Testing the k-means algorithm on the R2 dataset.

In [21]:
k_means_model = k_means(K = 10, max_iter = 300)
k_means_predictions = k_means_model.predict(R2)

(1000, 32)
Starting algorithm
Finished iteration 0
Finished iteration 1


c:\users\j\appdata\local\programs\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\j\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finished iteration 2
Finished iteration 3
Finished iteration 4
Finished iteration 5
Finished iteration 6
Finished iteration 7
Finished iteration 8
Finished iteration 9
Finished iteration 10
Finished iteration 11
Finished iteration 12
Finished iteration 13
Finished iteration 14
Finished iteration 15
Finished iteration 16
Finished iteration 17
Finished iteration 18
Finished iteration 19
Finished iteration 20
Finished iteration 21
Finished iteration 22
Finished iteration 23
Finished iteration 24
Finished iteration 25
Finished iteration 26
Finished iteration 27
Finished iteration 28
Finished iteration 29
Finished iteration 30
Finished iteration 31
Finished iteration 32
Finished iteration 33
Finished iteration 34
Finished iteration 35
Finished iteration 36
Finished iteration 37
Finished iteration 38
Finished iteration 39
Finished iteration 40
Finished iteration 41
Finished iteration 42
Finished iteration 43
Finished iteration 44
Finished iteration 45
Finished iteration 46
Finished iteration

Time to evaluate the k-means results on the new dataset.

In [22]:
k_means_predictions = k_means_predictions.astype(int)

k_means_purity = calculate_purity(k_means_predictions, 10, train_subset_labels)
print("K_means with R2 purity \n")
for i in range(len(k_means_purity)):
    print(class_names[i] + " purity: " + str(k_means_purity[i]))

ValueError: max() arg is an empty sequence

The error occurs because one category was not matched to a single vector.

In [23]:
print("K_means with R2 f-measure \n")
calculate_F_measure(k_means_predictions, 10, train_subset_labels)

K_means with R2 f-measure 



ValueError: max() arg is an empty sequence

Implementing a function which computes the Kullback-Leibler distance between two vectors.

In [24]:
def kullback_leibler_distance(P, Q):
    P_probabilities = np.zeros((len(P)))
    Q_probabilities = np.zeros((len(P)))
    P_sum = np.sum(P)
    Q_sum = np.sum(Q)
    
    for i in range(len(P)):
        P_probabilities[i] = P[i] / P_sum
        Q_probabilities[i] = Q[i] / Q_sum
        
    distance = 0
    
    for i in range(len(P_probabilities)):

        if P_probabilities[i] == 0 or Q_probabilities[i] == 0:
            continue
        else:
            distance += (P_probabilities[i] - Q_probabilities[i]) * log(P_probabilities[i] /  Q_probabilities[i], 10)
            
    return distance

Trying out the function.

In [25]:
print(kullback_leibler_distance(R2[15], R2[12]))

0.5264454858943826


Bellow I implement the k_medoids algorithm. 

In [26]:
# max_iter is the maximum iterations that we allow in case we don't converge earlier
# k = 10 for k clusters (since the clothing categories are actually 10)

class k_medoids:
    def __init__(self, K = 10, max_iter = 300, tolerance = 0.001):
        self.K = K
        self.max_iter = max_iter
        self.tolerance = tolerance
        
        # list of sample indices for each cluster
        self.clusters = [[] for i in range(self.K)]
        
        # medoid point for each cluster
        self.medoids = []
        
    def predict(self, X):
        self.X = X
        print(X.shape)
        self.n_samples, self.n_features = X.shape
        print("Starting algorithm")
        
        # initialise medoids (10 of the histogram representations will be picked randomly)
        random_sample_indices = np.random.choice(self.n_samples, self.K, replace=False)
        self.medoids = [self.X[index] for index in random_sample_indices]
        
        # optimisation
        
        for i in range(self.max_iter):
            # update clusters
            self.clusters = self._create_clusters(self.medoids)
            
            # update centroids
            medoids_old = self.medoids # in order to check for convergence
            self.medoids, change = self._get_medoids(self.clusters)
            
            # check if medoids changed
            if not change:
                break
            
            print("Finished iteration", i)
            
        # return cluster labels
        return self.__get_cluster_labels(self.clusters)
    
    def __get_cluster_labels(self, clusters):
        labels = np.empty(self.n_samples)
        for cluster_index, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_index
                
        return labels
       
    # assign each sample to the cluster of the closest medoid
    def _create_clusters(self, medoids):
        clusters = [[] for i in range(self.K)]
        
        for index, sample in enumerate(self.X):
            medoid_index = self._closest_medoid(sample, medoids)
            clusters[medoid_index].append(index)
            
        return clusters
    
    # calculate the distances of the current sample from each medoid (according to Kullback-Leibler distance)
    def _closest_medoid(self, sample, medoids):
        distances = [kullback_leibler_distance(sample, medoid) for medoid in medoids]
        closest_index = np.argmin(distances)
        return closest_index
    
    # calculate the centroids as the mean value of the samples in each cluster
    def _get_medoids(self, clusters):
        medoid_change = False
        medoids_new = np.zeros((self.K, self.n_features))
        old_distance_sum = self._calculate_distances(self.medoids, self.clusters)
        
        for cluster_index, cluster in enumerate(clusters):
            previous_dist = self._calculate_cluster_distance(self.medoids[cluster_index], cluster)
            best_sample_index = 0
            lesser_distance = 0 
            
            for sample_index, sample in enumerate(cluster):
                
                if (self.X[sample] != self.medoids[cluster_index]).all():
                    distance = self._calculate_cluster_distance(self.X[sample], cluster)
                    if distance < lesser_distance:
                        lesser_distance = distance
                        best_sample_index = sample_index
                        
            if lesser_distance < previous_dist:
                medoid_change = True
                medoids_new[cluster_index] = self.X[cluster[best_sample_index]]
            else:
                medoids_new[cluster_index] = self.medoids[cluster_index]
                
       
        return medoids_new, medoid_change
    
    # check for convergence
    def _is_converged(self, centroids_old, centroids):
        distances = [euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        return sum(distances) <= self.tolerance
    
    # calculate the sum of all distances of the samples from their cluster's medoid
    def _calculate_distances(self, medoids, clusters):
        distance = 0
        
        for cluster_index, cluster in enumerate(clusters):
            for sample in cluster:
                distance += kullback_leibler_distance(self.X[sample], medoids[cluster_index])
                
        return distance
        
    # calculate the sum of all distances of a sample in each cluster  
    def _calculate_cluster_distance(self, sample, cluster):
        distance = 0
        
        for item in cluster:
            distance += kullback_leibler_distance(sample, self.X[item])
                
        return distance

Predictions using the k-medoids algorithm.

In [27]:
k_medoids_model = k_medoids(K = 10, max_iter = 20, tolerance=0.001)
k_medoids_predictions = k_medoids_model.predict(R2)

(1000, 32)
Starting algorithm
Finished iteration 0
Finished iteration 1
Finished iteration 2
Finished iteration 3
Finished iteration 4
Finished iteration 5
Finished iteration 6
Finished iteration 7
Finished iteration 8
Finished iteration 9
Finished iteration 10
Finished iteration 11
Finished iteration 12
Finished iteration 13
Finished iteration 14
Finished iteration 15
Finished iteration 16
Finished iteration 17
Finished iteration 18
Finished iteration 19


In [28]:
print(k_medoids_predictions)

[5. 1. 0. 7. 6. 3. 4. 7. 3. 2. 6. 6. 0. 3. 6. 6. 5. 0. 6. 3. 1. 2. 1. 8.
 4. 1. 7. 0. 0. 9. 0. 8. 1. 0. 7. 3. 0. 0. 6. 1. 3. 6. 0. 6. 4. 1. 3. 3.
 3. 3. 1. 9. 2. 4. 0. 1. 7. 0. 6. 7. 0. 3. 1. 0. 2. 3. 9. 8. 3. 9. 8. 1.
 0. 3. 3. 9. 8. 0. 0. 0. 3. 8. 7. 0. 0. 6. 2. 3. 6. 3. 4. 2. 4. 1. 6. 6.
 4. 8. 7. 8. 6. 6. 0. 2. 6. 6. 2. 0. 0. 2. 6. 0. 0. 8. 3. 0. 3. 0. 0. 0.
 2. 4. 0. 0. 3. 0. 0. 0. 2. 6. 0. 3. 0. 2. 2. 0. 2. 2. 3. 7. 0. 2. 3. 2.
 0. 2. 2. 0. 3. 2. 3. 3. 3. 0. 3. 0. 0. 1. 0. 6. 0. 7. 0. 0. 0. 6. 6. 2.
 0. 0. 3. 2. 2. 0. 3. 2. 0. 0. 0. 0. 1. 3. 3. 6. 5. 6. 2. 1. 0. 4. 0. 0.
 7. 5. 3. 0. 2. 2. 7. 2. 6. 9. 5. 1. 3. 9. 4. 7. 4. 4. 0. 0. 0. 9. 5. 1.
 0. 7. 6. 5. 3. 6. 7. 3. 3. 8. 0. 0. 5. 9. 3. 9. 4. 0. 4. 4. 4. 3. 9. 7.
 4. 3. 4. 1. 3. 3. 9. 3. 3. 0. 8. 7. 9. 0. 3. 8. 5. 0. 8. 5. 7. 9. 7. 6.
 6. 7. 9. 6. 7. 6. 4. 5. 1. 0. 9. 4. 5. 1. 9. 6. 8. 3. 7. 8. 9. 1. 0. 8.
 9. 4. 4. 3. 0. 6. 9. 6. 4. 9. 9. 8. 3. 8. 0. 3. 0. 0. 0. 6. 9. 6. 0. 0.
 0. 3. 6. 3. 9. 3. 7. 3. 6. 2. 0. 2. 4. 0. 3. 9. 1.

In [29]:
k_medoids_predictions = k_medoids_predictions.astype(int)
print(k_medoids_predictions)

[5 1 0 7 6 3 4 7 3 2 6 6 0 3 6 6 5 0 6 3 1 2 1 8 4 1 7 0 0 9 0 8 1 0 7 3 0
 0 6 1 3 6 0 6 4 1 3 3 3 3 1 9 2 4 0 1 7 0 6 7 0 3 1 0 2 3 9 8 3 9 8 1 0 3
 3 9 8 0 0 0 3 8 7 0 0 6 2 3 6 3 4 2 4 1 6 6 4 8 7 8 6 6 0 2 6 6 2 0 0 2 6
 0 0 8 3 0 3 0 0 0 2 4 0 0 3 0 0 0 2 6 0 3 0 2 2 0 2 2 3 7 0 2 3 2 0 2 2 0
 3 2 3 3 3 0 3 0 0 1 0 6 0 7 0 0 0 6 6 2 0 0 3 2 2 0 3 2 0 0 0 0 1 3 3 6 5
 6 2 1 0 4 0 0 7 5 3 0 2 2 7 2 6 9 5 1 3 9 4 7 4 4 0 0 0 9 5 1 0 7 6 5 3 6
 7 3 3 8 0 0 5 9 3 9 4 0 4 4 4 3 9 7 4 3 4 1 3 3 9 3 3 0 8 7 9 0 3 8 5 0 8
 5 7 9 7 6 6 7 9 6 7 6 4 5 1 0 9 4 5 1 9 6 8 3 7 8 9 1 0 8 9 4 4 3 0 6 9 6
 4 9 9 8 3 8 0 3 0 0 0 6 9 6 0 0 0 3 6 3 9 3 7 3 6 2 0 2 4 0 3 9 1 0 8 1 4
 6 6 2 0 0 0 0 6 8 0 0 7 6 4 0 0 9 9 6 3 3 2 7 7 0 8 3 0 3 9 1 6 2 2 7 2 3
 2 2 4 7 3 1 1 0 0 0 0 6 5 0 2 2 2 2 0 6 9 4 0 0 0 8 1 0 9 8 4 3 6 3 6 3 0
 0 4 3 9 1 8 8 9 3 6 3 3 3 3 7 0 6 7 0 5 6 4 9 8 6 6 1 6 6 0 3 4 0 9 3 8 6
 3 6 0 4 3 2 6 5 2 0 6 6 7 3 8 8 9 9 0 8 6 8 3 0 6 3 3 4 7 0 7 0 6 3 4 3 0
 0 3 0 3 6 5 3 7 0 6 0 6 

Evaluating the k-medoids result using the functions I implemented earlier.

In [30]:
k_medoids_purity = calculate_purity(k_medoids_predictions, 10, train_subset_labels)
print("K_medoids purity \n")
for i in range(len(k_medoids_purity)):
    print(class_names[i] + " purity: " + str(k_medoids_purity[i]))

[1, 0, 7, 6, 2, 4, 9, 3, 8, 5]
K_medoids purity 

T-shirt/top purity: 0.12
Trouser purity: 0.39
Pullover purity: 0.14
Dress purity: 0.06
Coat purity: 0.03
Sandal purity: 0.03
Shirt purity: 0.34
Sneaker purity: 0.27
Bag purity: 0.15
Ankle boot purity: 0.26


In [31]:
purity = np.mean(k_medoids_purity)
print(purity)

0.179


In [32]:
print("K_medoids f-measure \n")
calculate_F_measure(k_medoids_predictions, 10, train_subset_labels)

K_medoids f-measure 

[1, 0, 7, 6, 2, 4, 9, 3, 8, 5]
Cluster 0  f-measure: 0.22285714285714286
Cluster 1  f-measure: 0.15894039735099338
Cluster 2  f-measure: 0.2621359223300971
Cluster 3  f-measure: 0.26877470355731226
Cluster 4  f-measure: 0.2545454545454546
Cluster 5  f-measure: 0.060000000000000005
Cluster 6  f-measure: 0.22510822510822512
Cluster 7  f-measure: 0.10084033613445377
Cluster 8  f-measure: 0.189873417721519
Cluster 9  f-measure: 0.05769230769230769
Method total f-measure: 1.8007679072975058


Bellow I implement a class for the hierarchical divisive clustering algorithm.

In [33]:
class hierarchical_divisive_clustering:
    def __init__(self, K = 10):
        self.K = K
        
        # in hierarchical divisive clustering we start with only one cluster for the whole set
        self.clusters = []
        
    def predict(self, X):
        self.X = X
        self.n_samples, self.n_features = X.shape
        self.clusters.append([index for index in range(self.n_samples)])
       
        divCounter = 1
        while len(self.clusters) < self.K: # keep dividing
            print("Division ", divCounter)
            
            # calculate max variance of each cluster in order to decide which to split
            cluster_to_split_index = self.max_variance_criterion(self.clusters)
            
            # remove the cluster to be splitted from the list, split it and add the two new clusters to the list
            cluster_to_split = self.clusters.pop(cluster_to_split_index)
            newCluster1, newCluster2 = self.split_cluster(cluster_to_split)
            self.clusters.append(newCluster1)
            self.clusters.append(newCluster2)
            
            divCounter += 1
            
        labels = self.__get_cluster_labels(self.clusters)
        return labels
            
    def max_variance_criterion(self, clusters):
        variances = np.zeros(len(clusters))

        for i in range(len(clusters)):
            variances[i] = self.calculate_cluster_variance(clusters[i])

        index = np.argmax(variances)

        return index

    def calculate_cluster_variance(self, cluster):
        mean_value = np.zeros(self.n_features)

        for i in range(len(cluster)):
            mean_value = np.add(mean_value, self.X[cluster[i]])

        if(len(cluster) > 0):
            mean_value = mean_value / len(cluster)

        all_features = []

        for i in range(len(cluster)):
            all_features.append(self.X[cluster[i]])

        deviations = [np.subtract(x, mean_value) ** 2 for x in all_features]

        variance = np.zeros(self.n_features)

        for i in range(len(deviations)):
            variance = np.add(variance, deviations[i])

        if(len(cluster) > 0):
            return np.sum(variance / len(cluster))
        
        return 0



    def __get_cluster_labels(self, clusters):
        labels = np.empty(self.n_samples)
        for cluster_index, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_index

        return labels

    def split_cluster(self, cluster):
        distances = np.zeros((self.n_samples, self.n_samples))

        for i in range(len(cluster)):
            for j in range(len(cluster)):
                for n in range(len(cluster)):
                    dist1 = euclidean_distance(self.X[cluster[n]], self.X[cluster[i]])
                    dist2 = euclidean_distance(self.X[cluster[n]], self.X[cluster[j]])
                    if dist1 <= dist2:
                        distances[i][j] = np.add(distances[i][j], dist1)
                    else:
                        distances[i][j] = np.add(distances[i][j], dist2)

        (x1, x2) = np.unravel_index(distances.argmin(), distances.shape)       
        cluster1 = []
        cluster2 = []

        for i in range(len(cluster)):
            if (euclidean_distance(self.X[cluster[i]], self.X[x1]) <= euclidean_distance(self.X[cluster[i]], self.X[x2])):
                cluster1.append(cluster[i])
            else:
                cluster2.append(cluster[i])

        return cluster1, cluster2

Here I will reduce the size of the dataset even further (200 samples), because the algorithm I implemented was slow because of the triple for-loop I implement for the choice of samples on the base of which the cluster will be divided.

In [34]:
train_subset_size = 200
train_subset_images = []
train_subset_labels = []

for category_index in range(10):
    remaining_from_category = train_subset_size / 10 # we need equally distributed data per category
    image_index = 0
    
    while remaining_from_category > 0 :
        if train_labels[image_index] == category_index:
            train_subset_images.append(train_images[image_index])
            train_subset_labels.append(train_labels[image_index])
            remaining_from_category -= 1
        image_index += 1
        
# R1 dataset : 28x28 = 784 feature vector

train_subset_images = np.array(train_subset_images)
train_subset_labels = np.array(train_subset_labels)

nsamples, nx, ny = train_subset_images.shape 
train_subset_images_R1 = train_subset_images.reshape((nsamples,nx*ny))

In [35]:
hierarchical_divisive_clustering_model = hierarchical_divisive_clustering(K = 10)
hierarchical_divisive_clustering_predictions = hierarchical_divisive_clustering_model.predict(train_subset_images_R1)

Division  1
Division  2
Division  3
Division  4
Division  5
Division  6
Division  7
Division  8
Division  9


In [36]:
hierarchical_divisive_clustering_predictions = hierarchical_divisive_clustering_predictions.astype(int)
print(hierarchical_divisive_clustering_predictions)


[8 3 2 1 1 8 3 1 3 3 1 3 3 1 8 3 0 8 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 2 8 8 8 3 0 0 3 8 3 3 8 0 8 0 0 3 8 8 8 0 3 2 3 3 2 2 3 3 3 3 0 2 3 3
 3 3 3 8 3 3 3 3 0 0 0 0 3 3 3 0 0 3 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 8 0 3 0 0 8 3 3 8 0 3 0 3 3 3 0 8 3 8 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 8 3 3 0 0 0 0 0 3 8 3 3 0 3 0 0 3 3 3 8 0 0 3 3 0
 3 8 8 3 3 3 3 3 0 3 3 8 3 3 3]


Again, there are clothing categories to which not even one datapoint was matched to, so an error occurs on the stats calculation.

In [37]:
calculate_purity(hierarchical_divisive_clustering_predictions, 10, train_subset_labels)

ValueError: max() arg is an empty sequence

In total, the k-means algorithm on the first dataset achieved a purity score of 0.25 and an f-score of 3.05, whereas the k-medoids algorithm (R2-set) achieved a purity score of 0.179 and an f-score of 1.8. 